In [1]:
#sparksession

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("spark_job").master("local[*]").getOrCreate()
spark


#local[*] takes all the cores present in the machine, you cant specify number of executors in local mode
#For local development, Spark runs in a single executor but can process tasks in parallel across multiple cores.

In [2]:
sc = spark.sparkContext
print("Number of cores:", sc.defaultParallelism) 

#it gives number of cores in the machine as i used local[*]

Number of cores: 10


In [10]:
#reading parquet files
df_transactions = spark.read.parquet("spark-experiments/data/data_skew/transactions.parquet")

In [12]:
df_transactions.rdd.getNumPartitions()

12

In [13]:
df_transactions.show()

+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+------------+
|   cust_id|start_date|  end_date|         txn_id|      date|year|month|day| expense_type|   amt|        city|
+----------+----------+----------+---------------+----------+----+-----+---+-------------+------+------------+
|C0YDPQWPBJ|2010-07-01|2018-12-01|TZ5SMKZY9S03OQJ|2018-10-07|2018|   10|  7|Entertainment| 10.42|      boston|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYIAPPNU066CJ5R|2016-03-27|2016|    3| 27| Motor/Travel| 44.34|    portland|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TETSXIK4BLXHJ6W|2011-04-11|2011|    4| 11|Entertainment|  3.18|     chicago|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TQKL1QFJY3EM8LO|2018-02-22|2018|    2| 22|    Groceries|268.97| los_angeles|
|C0YDPQWPBJ|2010-07-01|2018-12-01|TYL6DFP09PPXMVB|2010-10-16|2010|   10| 16|Entertainment|  2.66|     chicago|
|C0YDPQWPBJ|2010-07-01|2018-12-01|T1SMX9EUG21BBSE|2015-02-11|2015|    2| 11|    Education| 54.14|    portland|
|